In [1]:
import json
import pandas as pd
from pandas import DataFrame,Series
import matplotlib.pyplot as plt
import pymongo
import plotly.plotly as py
import plotly.graph_objs as go
import cufflinks as cf

In [2]:
connection = pymongo.MongoClient() # connecting to mongo
db = connection.twitterdb

In [3]:
tweets = db.final_tweets3.find({}, {"text": 1, "polarity": 1, "subjectivity": 1, "filter": 1,"Bin": 1, "created_at": 1})

In [4]:
l = [t for t in tweets]

df = DataFrame(l)

In [5]:
df[:5]

,Bin,_id,created_at,filter,polarity,subjectivity,text
0,Labour,59145d1558e3c0228c6c940a,Thu May 11 12:46:13 +0000 2017,Labour,0.0,0.0,RT @guardian: The Snap: Labour's leaked manife...
1,Labour,59145d1658e3c0228c6c940c,Thu May 11 12:46:13 +0000 2017,Jeremy Corbyn,0.8,1.0,Are you happy to have Branson's Virgin Care ra...
2,Conservative,59145d1658e3c0228c6c940e,Thu May 11 12:46:13 +0000 2017,conservative,0.0,0.0,RT @InxsyS: Trump is a conservative authoritar...
3,Labour,59145d1658e3c0228c6c9410,Thu May 11 12:46:13 +0000 2017,Labour,0.0,0.0,RT @JBeville_: Jez's Reds on June 8th #VoteLab...
4,Labour,59145d1658e3c0228c6c9412,Thu May 11 12:46:13 +0000 2017,Labour,0.0,0.0,Labour’s leaked manifesto is already sending t...


In [6]:
df['Bin'].value_counts()

Labour          665407
Conservative    213230
Neutral         125440
SNP              75078
UKIP             45382
LibDem           14838
Green             5717
other             4330
Cymru             1333
Name: Bin, dtype: int64

In [7]:
df2 = DataFrame(df['Bin'].value_counts())

In [8]:
df2

,Bin
Labour,665407
Conservative,213230
Neutral,125440
SNP,75078
UKIP,45382
LibDem,14838
Green,5717
other,4330
Cymru,1333


In [9]:
df3 = df.groupby('Bin').mean()

In [10]:
df3['Bin_size'] = df['Bin'].value_counts()

In [11]:
df3

,polarity,subjectivity,Bin_size
Bin,,,
Conservative,0.039961,0.308564,213230
Cymru,0.106627,0.300329,1333
Green,-0.065610,0.346105,5717
Labour,0.071209,0.318548,665407
LibDem,0.103378,0.315196,14838
Neutral,0.097851,0.300357,125440
SNP,0.090674,0.328542,75078
UKIP,0.077404,0.278568,45382
other,0.088636,0.280607,4330


In [12]:
df3['name'] = df3.index

In [13]:
df3

,polarity,subjectivity,Bin_size,name
Bin,,,,
Conservative,0.039961,0.308564,213230,Conservative
Cymru,0.106627,0.300329,1333,Cymru
Green,-0.065610,0.346105,5717,Green
Labour,0.071209,0.318548,665407,Labour
LibDem,0.103378,0.315196,14838,LibDem
Neutral,0.097851,0.300357,125440,Neutral
SNP,0.090674,0.328542,75078,SNP
UKIP,0.077404,0.278568,45382,UKIP
other,0.088636,0.280607,4330,other


In [20]:
df4 = df3.filter(items=['Conservative', 'Green', 'Labour', 'LibDem', 'SNP', 'UKIP'], axis=0)

In [137]:
df5 = df4.filter(items=['Conservative', 'Labour', 'LibDem', 'SNP', 'UKIP'], axis=0)

In [138]:
df5

,polarity,subjectivity,Bin_size,name,party_colour
Bin,,,,,
Conservative,0.039961,0.308564,213230,Conservative,"rgb(65,105,225)"
Labour,0.071209,0.318548,665407,Labour,"rgb(139,0,0)"
LibDem,0.103378,0.315196,14838,LibDem,"rgb(255,165,0)"
SNP,0.090674,0.328542,75078,SNP,"rgb(255,255,0)"
UKIP,0.077404,0.278568,45382,UKIP,"rgb(148,0,211)"


In [77]:
df4

,polarity,subjectivity,Bin_size,name,party_colour
Bin,,,,,
Conservative,0.039961,0.308564,213230,Conservative,"rgb(65,105,225)"
Green,-0.065610,0.346105,5717,Green,"rgb(34,139,34)"
Labour,0.071209,0.318548,665407,Labour,"rgb(139,0,0)"
LibDem,0.103378,0.315196,14838,LibDem,"rgb(255,165,0)"
SNP,0.090674,0.328542,75078,SNP,"rgb(255,255,0)"
UKIP,0.077404,0.278568,45382,UKIP,"rgb(148,0,211)"


In [119]:

for row in df4.index.values:
    print row
       

Conservative
Green
Labour
LibDem
SNP
UKIP


In [25]:
df4['party_colour'] = ['rgb(65,105,225)','rgb(34,139,34)', 'rgb(139,0,0)', 'rgb(255,165,0)','rgb(255,255,0)' ,
'rgb(148,0,211)']

In [88]:
df4['polarity']

Bin
Conservative    0.039961
Green          -0.065610
Labour          0.071209
LibDem          0.103378
SNP             0.090674
UKIP            0.077404
Name: polarity, dtype: float64

In [139]:
fig = {
    'data': [
                {
            'x': df4.loc[party].polarity, 
            'y': df4.loc[party].subjectivity,
            'name' : party,
            'mode' : 'markers',
            
            'marker' : {
                'color' : df4.loc[party].party_colour,
                'size' : df4.loc[party].Bin_size/5000,
                'opacity' : 0.6
            }
                        
        } for party in df5.index.values 
        ],
    'layout': {
        'xaxis': {'title': 'Polarity'},
        'yaxis': {'title': 'Subjectivity'}
    }
}

In [141]:
py.iplot(fig)

In [99]:
figure = df4.iplot(kind='bubble', x='polarity', y='subjectivity',  size='Bin_size', 
             xTitle='Polarity', yTitle='Subjectivity', text ='name' )

In [95]:
figure

In [ ]:
df4 = cf.datagen.bubble()

py.iplot([{
    'x': df.index,
    'y': df[col],
    'name': col
}  for col in df.columns], filename='cufflinks/simple-line')